In [1]:
from google.colab import drive
import os

drive.mount('/content/drive')
# Cài đặt Unsloth và các thư viện cần thiết
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-889sv5d5/unsloth_d97fc6f8af5949e5b968d53cda77df80
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-889sv5d5/unsloth_d97fc6f8af5949e5b968d53cda77df80
  Resolved https://github.com/unslothai/unsloth.git to commit a2d7811fe8283475f50c88645fb5d124dedb714c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.24.0-py3-none-any.whl (423 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.8.6
    Uninstalling trl-0.8.6:
      Successfully uninstalled trl-0.8.6
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl 

In [2]:
# Cấu hình đường dẫn
FILES = {
    "train": "/content/drive/MyDrive/NLP/data_llm/train_llm.jsonl",
    "dev": "/content/drive/MyDrive/NLP/dev_llm_v2.jsonl",
    "test": "/content/drive/MyDrive/NLP/test_llm_v2.jsonl"
}

DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/NLP/unsloth_qwen_absa"

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

# 1. Load Model Qwen 2.5
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 2. Gắn LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Qwen2 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from datasets import load_dataset

# Load dữ liệu
dataset = load_dataset("json", data_files={"train": FILES["train"], "validation": FILES["dev"]})

# Format Prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Map dữ liệu
dataset = dataset.map(formatting_prompts_func, batched = True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7785 [00:00<?, ? examples/s]

Map:   0%|          | 0/1112 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from transformers.trainer_utils import get_last_checkpoint
import os

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["validation"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,

    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        num_train_epochs = 2,
        output_dir = DRIVE_OUTPUT_DIR,
        save_strategy = "steps",
        save_steps = 100,                # Lưu mỗi 100 bước
        save_total_limit = 2,            # Chỉ giữ 2 bản mới nhất trên Drive
        eval_strategy = "no",
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

last_checkpoint = get_last_checkpoint(DRIVE_OUTPUT_DIR)

if last_checkpoint:
    print(f"Phát hiện checkpoint cũ trên Drive: {last_checkpoint}")
    print("Đang khôi phục và tiếp tục train...")
    trainer.train(resume_from_checkpoint=True)
else:
    print("Không tìm thấy checkpoint trên Drive. Bắt đầu train mới...")
    trainer.train()

Map (num_proc=2):   0%|          | 0/7785 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1112 [00:00<?, ? examples/s]

Phát hiện checkpoint cũ trên Drive: /content/drive/MyDrive/NLP/unsloth_qwen_absa/checkpoint-1100
Đang khôi phục và tiếp tục train...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,785 | Num Epochs = 2 | Total steps = 1,948
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1110,0.279800
1120,0.305000
1130,0.281900
1140,0.272500
1150,0.315100
1160,0.314500
1170,0.296900
1180,0.291100
1190,0.273800
1200,0.305500


In [ ]:
# 1. Áp dụng format "Alpaca" giống hệt tập Train
val_dataset = dataset["validation"].map(formatting_prompts_func, batched=True)

# 2. Tokenize cột "text" (Biến chữ thành số)
def tokenize_for_validation(examples):
    # Tokenize cột "text" vừa tạo ra
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=2048)
val_dataset = val_dataset.map(tokenize_for_validation, batched=True)

# 3. Dọn dẹp cột thừa & Set format
columns_to_keep = ['input_ids', 'attention_mask']
columns_to_remove = [col for col in val_dataset.column_names if col not in columns_to_keep]
val_dataset = val_dataset.remove_columns(columns_to_remove)

val_dataset.set_format("torch")

Map:   0%|          | 0/1112 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

def run_full_dev(model, val_data):
    print(f"TÍNH LOSS TRÊN TẬP DEV ({len(val_data)} mẫu)")
    dev_loader = DataLoader(val_data, batch_size=4, shuffle=False)
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in tqdm(dev_loader):
            input_ids = batch['input_ids'].to("cuda")
            attention_mask = batch['attention_mask'].to("cuda")
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            total_loss += outputs.loss.item()

    avg_loss = total_loss / len(dev_loader)
    print(f"Validation Loss: {avg_loss:.4f}")
    return avg_loss

# Gọi hàm
run_full_dev(model, val_dataset)

TÍNH LOSS TRÊN TẬP DEV (1112 mẫu)


100%|██████████| 278/278 [31:29<00:00,  6.80s/it]

Validation Loss: 11.2163


11.216254755747403

In [ ]:
import os
save_path = "/content/drive/MyDrive/NLP/output_source2"
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Lưu model và tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Đã lưu xong model vào: {save_path} ...")

Đã lưu xong model vào: /content/drive/MyDrive/NLP/output_source2 ...


In [ ]:
from unsloth import FastLanguageModel
import random

FastLanguageModel.for_inference(model)

# Định nghĩa lại Prompt cho lúc Test (Chỉ đưa Instruction + Input, bỏ Output)
alpaca_inference_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
"""

def preview_dev_predictions(model, raw_dataset, num_samples=20):
    print(f"KIỂM TRA KẾT QUẢ TRÊN {num_samples} MẪU NGẪU NHIÊN TỪ TẬP DEV")

    # Lấy ngẫu nhiên từ tập dữ liệu gốc (lúc chưa map/tokenize)
    indices = random.sample(range(len(raw_dataset)), min(num_samples, len(raw_dataset)))

    for i, idx in enumerate(indices):
        item = raw_dataset[idx]
        # Chuẩn bị input
        instruction = item['instruction']
        input_text = item['input']
        ground_truth = item['output'] # Đáp án gốc

        # Tạo prompt chỉ đến đoạn "Response:"
        prompt = alpaca_inference_prompt.format(instruction, input_text)

        # Tokenize và đưa vào GPU
        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

        # Model sinh văn bản
        outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)

        # Decode kết quả: cắt bỏ phần prompt đi, chỉ lấy phần mới sinh ra
        generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        # Tách lấy phần trả lời sau chữ "Response:"
        response_start = generated_text.find("### Response:") + len("### Response:")
        prediction = generated_text[response_start:].strip()

        print(f"Mẫu #{i+1}")
        print(f"Input: {input_text}")
        print(f"Gốc (Ref): {ground_truth}")
        print(f"Model đoán: {prediction}")

preview_dev_predictions(model, dataset["validation"])

KIỂM TRA KẾT QUẢ TRÊN 20 MẪU NGẪU NHIÊN TỪ TẬP DEV
Mẫu #1
Input: Mới hốt em nó sáng nay , vô cùng đẹp luôn các bác ạ , tuy ko biết sử dụng 1 thời gian ra sao , nhưng em tết camera thì vẫn còn mờ , nhòa . còn lại đều 5 sao hết :)
Gốc (Ref): [{"aspect": "DESIGN", "sentiment": "POSITIVE", "span": "vô cùng đẹp luôn"}, {"aspect": "CAMERA", "sentiment": "NEGATIVE", "span": "em tết camera thì vẫn còn mờ , nhòa"}, {"aspect": "GENERAL", "sentiment": "POSITIVE", "span": "còn lại đều 5 sao hết"}]
Model đoán: [{"aspect": "DESIGN", "sentiment": "POSITIVE", "span": "vô cùng đẹp luôn"}, {"aspect": "CAMERA", "sentiment": "NEGATIVE", "span": "camera thì vẫn còn mờ , nhóa"}]
Mẫu #2
Input: Mua dc 3 thang.Đang dt zalo, chơi game, xem youtube bị văng ra bởi vì quảng cáo hay thông báo. Nản hết sức. Chỉnh tắt hết thông báo cung k dc . Đem ra TGDD roi cung k an thua.k nên mua
Gốc (Ref): [{"aspect": "FEATURES", "sentiment": "NEGATIVE", "span": "Đang dt zalo, chơi game, xem youtube bị văng ra bởi vì quảng cáo h

In [ ]:
import json
import os
from tqdm import tqdm
from datasets import load_dataset

test_file_path = FILES["test"]
# Đường dẫn lưu kết quả trên Drive
output_path = "/content/drive/MyDrive/NLP/predictions_source2.jsonl"
# Số câu mỗi lần lưu (Checkpoint)
SAVE_EVERY = 20

# Load dữ liệu thô
data_files = {"test": test_file_path}
test_dataset = load_dataset("json", data_files=data_files)["test"]
print(f"Tổng số câu cần test: {len(test_dataset)}")

FastLanguageModel.for_inference(model)

# Prompt chỉ có Instruction + Input
alpaca_inference_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
"""

def run_test_with_resume(model, tokenizer, dataset, output_file, batch_size=20):
    # KIỂM TRA ĐIỂM XUẤT PHÁT (RESUME)
    start_index = 0
    if os.path.exists(output_file):
        with open(output_file, 'r', encoding='utf-8') as f:
            # Đếm số dòng đã chạy
            start_index = sum(1 for _ in f)
        print(f"Tiếp tục chạy từ câu số: {start_index}")
    else:
        # Tạo thư mục cha nếu chưa có
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        print(f"Tạo file kết quả mới tại: {output_file}")

    if start_index >= len(dataset):
        print("Đã chạy xong")
        return
    buffer = [] # Bộ nhớ đệm để chứa kết quả tạm

    # Progress bar bắt đầu từ start_index
    progress_bar = tqdm(range(start_index, len(dataset)), initial=start_index, total=len(dataset))
    for i in progress_bar:
        item = dataset[i]

        # 1. Tạo Prompt
        prompt = alpaca_inference_prompt.format(item['instruction'], item['input'])

        # 2. Chạy Model
        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)

        # 3. Xử lý kết quả (Cắt bỏ prompt, chỉ lấy phần model trả lời)
        full_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        response_start = full_text.find("### Response:") + len("### Response:")
        prediction = full_text[response_start:].strip()

        # 4. Lưu vào buffer (Lưu cả input gốc để dễ tra cứu)
        result_obj = {
            "id": i, # Đánh số thứ tự để dễ quản lý
            "input": item['input'],
            "prediction": prediction
        }
        buffer.append(result_obj)

        # Nếu đầy buffer (đủ 20 câu) hoặc là câu cuối cùng
        if len(buffer) >= SAVE_EVERY or i == len(dataset) - 1:
            with open(output_file, 'a', encoding='utf-8') as f:
                for entry in buffer:
                    # Ghi từng dòng (JSON Lines format)
                    json.dump(entry, f, ensure_ascii=False)
                    f.write('\n')

            # Thông báo
            progress_bar.set_description(f"Đã lưu đến câu {i+1} vào Drive")
            buffer = [] # Xóa buffer để giải phóng RAM

    print(f"\n HOÀN THÀNH! File lưu tại: {output_file}")

run_test_with_resume(model, tokenizer, test_dataset, output_path, batch_size=SAVE_EVERY)

Generating test split: 0 examples [00:00, ? examples/s]

Tổng số câu cần test: 2225
Tiếp tục chạy từ câu số: 1800


Đã lưu đến câu 2225 vào Drive: 100%|██████████| 2225/2225 [3:22:11<00:00, 28.55s/it]


 HOÀN THÀNH! File lưu tại: /content/drive/MyDrive/NLP/predictions_source2.jsonl


In [ ]:
import json
import re
import pandas as pd
from collections import defaultdict
from datasets import load_dataset

PRED_FILE = "/content/drive/MyDrive/NLP/predictions_source2.jsonl"
TEST_FILE_PATH = FILES["test"]

# Danh sách nhãn chuẩn
ASPECTS = ["SCREEN", "CAMERA", "FEATURES", "BATTERY", "PERFORMANCE", "STORAGE", "DESIGN", "PRICE", "GENERAL", "SER&ACC"]
SENTIMENTS = ["POSITIVE", "NEGATIVE", "NEUTRAL"]

# HÀM EXTRACT & NORMALIZE
def extract_robust_triplets(text):
    triplets = []
    text = str(text)
    pattern = r'aspect"?: ["\'](.*?)["\'].*?sentiment"?: ["\'](.*?)["\']'
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
    for a, s in matches: triplets.append({"aspect": a, "sentiment": s})
    return triplets

def normalize_item(a, s):
    a = str(a).upper().strip()
    s = str(s).upper().strip()

    # Mapping đồng nghĩa
    if 'DISPLAY' in a or 'MÀN HÌNH' in a: a = 'SCREEN'
    if 'SOUND' in a or 'SPEAKER' in a or 'LOA' in a or 'WIFI' in a or 'SIM' in a or 'FACE ID' in a or 'VÂN TAY' in a: a = 'FEATURES'
    if 'MEMORY' in a or 'RAM' in a or 'CHIP' in a or 'LAG' in a or 'GAME' in a or 'MƯỢT' in a: a = 'PERFORMANCE'
    if 'ROM' in a or 'THẺ NHỚ' in a or 'DUNG LƯỢNG' in a or 'GB' in a: a = 'STORAGE'
    if 'SERVICE' in a or 'PHỤ KIỆN' in a or 'BẢO HÀNH' in a or 'NHÂN VIÊN' in a or 'GIAO HÀNG' in a or 'TAI NGHE' in a or 'SẠC' in a: a = 'SER&ACC'
    if 'MỎNG' in a or 'NHẸ' in a or 'ĐẸP' in a or 'CẦM' in a or 'LƯNG' in a or 'MÀU' in a: a = 'DESIGN'
    if 'TIỀN' in a or 'RẺ' in a or 'ĐẮT' in a: a = 'PRICE'

    # Mapping cảm xúc
    if 'TÍCH CỰC' in s or 'TỐT' in s or 'NGON' in s or 'OK' in s or 'LIKE' in s: s = 'POSITIVE'
    if 'TIÊU CỰC' in s or 'TỆ' in s or 'CHÁN' in s or 'LỖI' in s or 'BAD' in s: s = 'NEGATIVE'
    if 'TRUNG TÍNH' in s or 'BÌNH THƯỜNG' in s or 'ỔN' in s: s = 'NEUTRAL'

    return a, s

# HÀM TÍNH ĐIỂM
def calculate_metrics_final(pred_path, test_path):
    print("⏳ Đang load dữ liệu...")

    # A. Load file dự đoán
    predictions = {}
    with open(pred_path, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            # Lưu theo ID để map chính xác
            predictions[item['id']] = item['prediction']

    # B. Load file gốc
    dataset = load_dataset("json", data_files={"test": test_path})["test"]

    print(f"✅ Đã load: {len(predictions)} câu dự đoán / {len(dataset)} câu gốc.")

    # C. Khởi tạo biến đếm
    stats = {
        'aspect': {'micro': {'tp':0,'fp':0,'fn':0}, 'class': defaultdict(lambda: {'tp':0,'fp':0,'fn':0})},
        'polarity': {'micro': {'tp':0,'fp':0,'fn':0}, 'class': defaultdict(lambda: {'tp':0,'fp':0,'fn':0})},
        'strict': {'micro': {'tp':0,'fp':0,'fn':0}, 'class': defaultdict(lambda: {'tp':0,'fp':0,'fn':0})}
    }

    # D. Vòng lặp so khớp
    for i, item in enumerate(dataset):
        # Lấy dự đoán tương ứng với câu i
        # Nếu ko tìm thấy id thì bỏ qua
        if i not in predictions:
            continue

        pred_text = predictions[i]
        gold_text = item['output'] # Cột output trong file gốc

        # Extract
        pred_raw = extract_robust_triplets(pred_text)
        ref_raw = extract_robust_triplets(gold_text)

        # Chuẩn bị Set để so sánh
        p_asp, r_asp = [], []
        p_pol, r_pol = [], []
        p_str, r_str = [], []

        # Xử lý Prediction
        for p in pred_raw:
            a, s = normalize_item(p['aspect'], p['sentiment'])
            if a in ASPECTS and s in SENTIMENTS:
                p_asp.append(a)
                p_pol.append(s)
                p_str.append(f"{a}#{s}")

        # Xử lý Ground Truth
        for r in ref_raw:
            a, s = normalize_item(r['aspect'], r['sentiment'])
            if a in ASPECTS and s in SENTIMENTS:
                r_asp.append(a)
                r_pol.append(s)
                r_str.append(f"{a}#{s}")

        # Cập nhật thống kê
        def update_stats_internal(mode, p_items, r_items):
            p_set, r_set = set(p_items), set(r_items)
            tp = p_set.intersection(r_set)
            fp = p_set - r_set
            fn = r_set - p_set

            stats[mode]['micro']['tp'] += len(tp)
            stats[mode]['micro']['fp'] += len(fp)
            stats[mode]['micro']['fn'] += len(fn)

            for l in p_set.union(r_set):
                if l in tp: stats[mode]['class'][l]['tp'] += 1
                elif l in fp: stats[mode]['class'][l]['fp'] += 1
                elif l in fn: stats[mode]['class'][l]['fn'] += 1

        update_stats_internal('aspect', p_asp, r_asp)
        update_stats_internal('polarity', p_pol, r_pol)
        update_stats_internal('strict', p_str, r_str)

    # E. Tính toán F1
    def get_f1(tp, fp, fn):
        p = tp / (tp + fp + 1e-9)
        r = tp / (tp + fn + 1e-9)
        f = 2*p*r / (p+r+1e-9)
        return f * 100

    results = []
    tasks = [('aspect', ASPECTS), ('polarity', SENTIMENTS), ('strict', [f"{a}#{s}" for a in ASPECTS for s in SENTIMENTS])]

    for task_name, valid_labels in tasks:
        # Micro
        mic = stats[task_name]['micro']
        mic_f1 = get_f1(mic['tp'], mic['fp'], mic['fn'])

        # Macro
        f1s = []
        for label in valid_labels:
            s = stats[task_name]['class'][label]
            f = get_f1(s['tp'], s['fp'], s['fn'])
            f1s.append(f)
        mac_f1 = sum(f1s) / len(valid_labels)

        results.append([task_name.title(), mic_f1, mac_f1])

    # F. In kết quả
    df = pd.DataFrame(results, columns=["Task", "F1_Micro", "F1_Macro"])
    print("\n" + "="*60)
    print("BẢNG KẾT QUẢ")
    print("="*60)
    print(df.round(2).to_markdown(index=False))

calculate_metrics_final(PRED_FILE, TEST_FILE_PATH)

⏳ Đang load dữ liệu...


Generating test split: 0 examples [00:00, ? examples/s]

✅ Đã load: 2225 câu dự đoán / 2225 câu gốc.

BẢNG KẾT QUẢ
| Task     |   F1_Micro |   F1_Macro |
|:---------|-----------:|-----------:|
| Aspect   |      71.61 |      65.94 |
| Polarity |      84.13 |      72.06 |
| Strict   |      62.47 |      43.05 |


In [ ]:
# CODE SOI LỖI STRICT
def analyze_strict_errors(pred_path, test_path, limit=10):
    print(f"--- ĐANG TÌM {limit} LỖI SAI ĐIỂN HÌNH ---")
    predictions = {}
    with open(pred_path, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            predictions[item['id']] = item['prediction']

    dataset = load_dataset("json", data_files={"test": test_path})["test"]
    count = 0

    for i, item in enumerate(dataset):
        if i not in predictions: continue

        pred_text = predictions[i]
        gold_text = item['output']

        # Lấy cặp Strict đã chuẩn hóa
        p_raw = extract_robust_triplets(pred_text)
        r_raw = extract_robust_triplets(gold_text)

        p_str = set([f"{normalize_item(x['aspect'], x['sentiment'])[0]}#{normalize_item(x['aspect'], x['sentiment'])[1]}" for x in p_raw])
        r_str = set([f"{normalize_item(x['aspect'], x['sentiment'])[0]}#{normalize_item(x['aspect'], x['sentiment'])[1]}" for x in r_raw])

        # Nếu sai khác
        if p_str != r_str:
            print(f"❌ Mẫu #{i}")
            print(f"   Input: {item['input']}")
            print(f"   Model đoán (Raw): {pred_text}")
            print(f"   Model hiểu (Norm): {p_str}")
            print(f"   Đáp án gốc       : {r_str}")
            print("-" * 50)
            count += 1
            if count >= limit: break

analyze_strict_errors(PRED_FILE, TEST_FILE_PATH)

--- ĐANG TÌM 10 LỖI SAI ĐIỂN HÌNH ---
❌ Mẫu #1
   Input: Mua cho mẹ sài nên củng không đòi hỏi gì nhiều, máy đẹp camera siêu ảo, thử chiến game củng ok,pin sài dc 2 ngày với luot wep xem fim, nhân viên tgdd an minh KG phục vụ qua nhiệt tình cho 5*
   Model đoán (Raw): [{"aspect": "DESIGN", "sentiment": "POSITIVE", "span": "máy đẹp"}, {"aspect": "CAMERA", "sentiment": "POSITIVE", "span": "camera siêu ảo"}, {"aspect": "PERFORMANCE", "sentiment": "POSITIVE", "span": "thử chiến game củng ok"}, {"aspect": "BATTERY", "sentiment": "POSITIVE", "span": "pin sài dc 2 ngày với luot wep xem fim"}]
   Model hiểu (Norm): {'DESIGN#POSITIVE', 'CAMERA#POSITIVE', 'PERFORMANCE#POSITIVE', 'BATTERY#POSITIVE'}
   Đáp án gốc       : {'SER&ACC#POSITIVE', 'PERFORMANCE#POSITIVE', 'BATTERY#POSITIVE', 'DESIGN#POSITIVE', 'CAMERA#POSITIVE'}
--------------------------------------------------
❌ Mẫu #2
   Input: Máy xài tốt, mượt, sạc rất nhanh, pin trâu, mình dùng tác vụ bình thường (zalo, fb, youtube) thì được 1 ngà

In [ ]:
import json
import re
import pandas as pd
from collections import defaultdict
from datasets import load_dataset

PRED_FILE = "/content/drive/MyDrive/NLP/predictions_source2.jsonl"
TEST_FILE_PATH = FILES["test"]

# Danh sách nhãn chuẩn
ASPECTS = ["SCREEN", "CAMERA", "FEATURES", "BATTERY", "PERFORMANCE", "STORAGE", "DESIGN", "PRICE", "GENERAL", "SER&ACC"]
SENTIMENTS = ["POSITIVE", "NEGATIVE", "NEUTRAL"]

# 1. HÀM TRÍCH XUẤT
def extract_robust_triplets_v2(text):
    triplets = []
    text = str(text)
    # Regex bắt cả 3 trường: Aspect, Sentiment, và Span
    # Pattern này chấp nhận thứ tự lung tung trong json
    # Parse JSON nếu có thể, nếu lỗi thì dùng Regex
    try:
        # Cố gắng parse JSON list chuẩn
        # Tìm đoạn [...] đầu tiên
        json_match = re.search(r'\[.*\]', text, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group(0))
            for item in data:
                triplets.append({
                    "aspect": str(item.get("aspect", "")),
                    "sentiment": str(item.get("sentiment", "")),
                    "span": str(item.get("span", ""))
                })
            return triplets
    except:
        pass

    # Fallback: Dùng Regex nếu JSON lỗi
    pattern = r'aspect"?: ["\'](.*?)["\'].*?sentiment"?: ["\'](.*?)["\'].*?span"?: ["\'](.*?)["\']'
    matches = re.findall(pattern, text, re.DOTALL | re.IGNORECASE)
    for a, s, sp in matches:
        triplets.append({"aspect": a, "sentiment": s, "span": sp})
    return triplets

# HÀM CHUẨN HÓA
def normalize_item_v2(a, s, span):
    a = str(a).upper().strip()
    s = str(s).upper().strip()
    span = str(span).lower().strip() # Span dùng để check từ khóa

    # A. SỬA LỖI DỰA TRÊN SPAN (Nội dung)
    # Ưu tiên sửa Aspect dựa trên từ khóa trong Span trước

    # 1. Lỗi "Cam đẹp" -> Design (Mẫu #6, #9) -> Sửa thành CAMERA
    if 'cam' in span or 'ảnh' in span or 'hình' in span or 'video' in span or 'selfie' in span:
        if 'màn hình' not in span and 'cấu hình' not in span: # Trừ trường hợp "màn hình"
            a = 'CAMERA'

    # 2. Lỗi "Cảm ứng" -> Performance (Mẫu #3) -> Sửa thành FEATURES
    if 'cảm ứng' in span or 'vân tay' in span or 'face id' in span or 'loa' in span or 'wifi' in span or 'sim' in span:
        a = 'FEATURES'

    # 3. Lỗi "1GB", "Dung lượng" -> Performance (Mẫu #10) -> Sửa thành STORAGE
    if 'gb' in span or 'dung lượng' in span or 'thẻ nhớ' in span or 'bộ nhớ' in span:
        a = 'STORAGE'

    # 4. Lỗi "Sạc nhanh", "Sạc" -> Ser&Acc (Mẫu #2) -> Sửa thành BATTERY
    # Uu tiên Battery nếu có chữ "pin"
    if 'pin' in span:
        a = 'BATTERY'

    # 5. Lỗi "Xài tốt", "Máy ngon" -> Performance (Mẫu #2) -> Sửa thành GENERAL
    if span in ['máy xài tốt', 'máy ngon', 'ok', 'tốt', 'ổn', 'tuyệt vời']:
        a = 'GENERAL'

    # B. MAPPING ASPECT THÔNG THƯỜNG
    if 'DISPLAY' in a or 'MÀN' in a: a = 'SCREEN'
    if 'HIỆU NĂNG' in a or 'CHIP' in a or 'TỐC ĐỘ' in a or 'GAME' in a or 'MƯỢT' in a or 'LAG' in a: a = 'PERFORMANCE'
    if 'THIẾT KẾ' in a or 'ĐẸP' in a or 'MỎNG' in a or 'MÀU' in a: a = 'DESIGN'
    if 'GIÁ' in a or 'TIỀN' in a: a = 'PRICE'
    if 'PHỤ KIỆN' in a or 'BẢO HÀNH' in a or 'NHÂN VIÊN' in a or 'GIAO' in a: a = 'SER&ACC'
    if 'CHUNG' in a or 'TỔNG QUAN' in a: a = 'GENERAL'

    # C. MAPPING SENTIMENT
    if 'TÍCH CỰC' in s or 'TỐT' in s or 'NGON' in s or 'OK' in s or 'LIKE' in s: s = 'POSITIVE'
    if 'TIÊU CỰC' in s or 'TỆ' in s or 'CHÁN' in s or 'LỖI' in s or 'KÉM' in s: s = 'NEGATIVE'
    if 'TRUNG TÍNH' in s or 'BÌNH THƯỜNG' in s or 'ỔN' in s or 'TẠM' in s: s = 'NEUTRAL'

    # Mẫu #7: "hối tiếc" -> Negative
    if 'hối tiếc' in span or 'tệ' in span or 'chán' in span or 'lỗi' in span or 'thất vọng' in span:
        s = 'NEGATIVE'

    return a, s

# 2. HÀM TÍNH ĐIỂM
def calculate_metrics_optimized(pred_path, test_path):
    predictions = {}
    with open(pred_path, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            predictions[item['id']] = item['prediction']

    dataset = load_dataset("json", data_files={"test": test_path})["test"]

    stats = {
        'aspect': {'micro': {'tp':0,'fp':0,'fn':0}, 'class': defaultdict(lambda: {'tp':0,'fp':0,'fn':0})},
        'polarity': {'micro': {'tp':0,'fp':0,'fn':0}, 'class': defaultdict(lambda: {'tp':0,'fp':0,'fn':0})},
        'strict': {'micro': {'tp':0,'fp':0,'fn':0}, 'class': defaultdict(lambda: {'tp':0,'fp':0,'fn':0})}
    }

    for i, item in enumerate(dataset):
        if i not in predictions: continue

        # Extract v2 (Lấy cả span)
        pred_raw = extract_robust_triplets_v2(predictions[i])
        ref_raw = extract_robust_triplets_v2(item['output'])

        p_asp, r_asp = [], []
        p_pol, r_pol = [], []
        p_str, r_str = [], []

        # Normalize v2 (Dùng span để sửa aspect)
        for p in pred_raw:
            a, s = normalize_item_v2(p['aspect'], p['sentiment'], p['span'])
            if a in ASPECTS and s in SENTIMENTS:
                p_asp.append(a); p_pol.append(s); p_str.append(f"{a}#{s}")

        for r in ref_raw:
            a, s = normalize_item_v2(r['aspect'], r['sentiment'], r['span'])
            if a in ASPECTS and s in SENTIMENTS:
                r_asp.append(a); r_pol.append(s); r_str.append(f"{a}#{s}")

        # Update stats logic
        def update(mode, p_items, r_items):
            p_set, r_set = set(p_items), set(r_items)
            tp = p_set.intersection(r_set)
            fp = p_set - r_set
            fn = r_set - p_set
            stats[mode]['micro']['tp'] += len(tp)
            stats[mode]['micro']['fp'] += len(fp)
            stats[mode]['micro']['fn'] += len(fn)
            for l in p_set.union(r_set):
                if l in tp: stats[mode]['class'][l]['tp'] += 1
                elif l in fp: stats[mode]['class'][l]['fp'] += 1
                elif l in fn: stats[mode]['class'][l]['fn'] += 1

        update('aspect', p_asp, r_asp)
        update('polarity', p_pol, r_pol)
        update('strict', p_str, r_str)

    # In kết quả
    def get_f1(tp, fp, fn):
        p = tp / (tp + fp + 1e-9)
        r = tp / (tp + fn + 1e-9)
        f = 2*p*r / (p+r+1e-9)
        return f * 100

    results = []
    tasks = [('aspect', ASPECTS), ('polarity', SENTIMENTS), ('strict', [f"{a}#{s}" for a in ASPECTS for s in SENTIMENTS])]

    for task_name, valid_labels in tasks:
        # Micro
        mic = stats[task_name]['micro']
        mic_f1 = get_f1(mic['tp'], mic['fp'], mic['fn'])
        # Macro
        f1s = []
        for label in valid_labels:
            s = stats[task_name]['class'][label]
            f = get_f1(s['tp'], s['fp'], s['fn'])
            f1s.append(f)
        mac_f1 = sum(f1s) / len(valid_labels)
        results.append([task_name.title(), mic_f1, mac_f1])

    df = pd.DataFrame(results, columns=["Task", "F1_Micro", "F1_Macro"])
    print("\n" + "="*60)
    print("BẢNG KẾT QUẢ")
    print("="*60)
    print(df.round(2).to_markdown(index=False))

calculate_metrics_optimized(PRED_FILE, TEST_FILE_PATH)


BẢNG KẾT QUẢ
| Task     |   F1_Micro |   F1_Macro |
|:---------|-----------:|-----------:|
| Aspect   |      74.79 |      71.02 |
| Polarity |      84.36 |      71.86 |
| Strict   |      65.73 |      46.83 |
